[Home](../README.md)

### Feature Engineering

This Jupyter Notepad is a selection of data engineering processes I applied to my  data before model training to maximise the performance of my machine learning model.

#### Load the required dependencies

In [ ]:
# Import frameworks
import pandas as pd
import re

####  Store the data as a local variable

The data frame is a Pandas object that structures your tabular data into an appropriate format. It loads the complete data in memory so it is now ready for preprocessing.

In [4]:
data_frame = pd.read_csv("wrangled_data.csv")

####  Deriving new variables from existing ones

##### Encoding categorical variables

Data Encoding converts textual data into numerical format, so that it can be used as input for algorithms to process. The reason for encoding is that most machine learning algorithms work with numbers and not with text or categorical variables.


##### 'rating' column encoding

This code encodes the 'rating' column by mapping movie ratings to a custom numerical order from least to most restrictive

In [5]:
# to find the unique ratings in the data
data_frame['rating'].unique()

array(['R', 'PG', 'G', 'Not Rated', 'NC-17', 'Approved', 'TV-PG', 'PG-13',
       'Unrated', 'X', 'TV-MA', 'TV-14'], dtype=object)

Using a predefined list, each rating is assigned an index and mapped to its corresponding value. This approach preserves the ordinal relationship between ratings (e.g., G < PG < PG-13 < R), making it more meaningful for ML models

In [ ]:
# Define the custom class order from least to most violent
rating_order = ['G', 'TV-PG', 'PG', 'Approved', 'Unrated', 'Not Rated', 'PG-13', 'TV-14', 'R', 'TV-MA', 'NC-17', 'X']

# Fit and transform the 'rating' column
data_frame['rating'] = pd.Series(data_frame['rating'])

# Map the 'rating' column to the custom order
data_frame['rating'] = data_frame['rating'].map({rating: idx for idx, rating in enumerate(rating_order)})

# Display the first few rows of the DataFrame to verify the encoding
print(data_frame[['rating']])

# Print the mapping of classes to labels
print(dict(enumerate(rating_order)))


      rating
0          8
1          8
2          2
3          2
4          8
...      ...
7569       5
7570       8
7571       6
7572       8
7573       5

[7574 rows x 1 columns]
{0: 'G', 1: 'TV-PG', 2: 'PG', 3: 'Approved', 4: 'Unrated', 5: 'Not Rated', 6: 'PG-13', 7: 'TV-14', 8: 'R', 9: 'TV-MA', 10: 'NC-17', 11: 'X'}


##### 'genre' column encoding

This code one-hot encodes the 'genre' column, creating separate binary columns for each genre. If a movie belongs to a specific genre, the corresponding column is set to 1; otherwise, it's 0. 

In [7]:
# One-hot encode the 'genre' column
data_frame = pd.get_dummies(data_frame, columns=['genre'])

# Display the first few rows of the encoded dataframe
print(data_frame.head())

                                             name  rating  year  \
0                                     The Shining       8  1980   
1                                 The Blue Lagoon       8  1980   
2  Star Wars: Episode V - The Empire Strikes Back       2  1980   
3                                       Airplane!       2  1980   
4                                      Caddyshack       8  1980   

        released     score     votes         director  \
0  June 13, 1980  0.878378  0.386241  Stanley Kubrick   
1   July 2, 1980  0.527027  0.027069   Randal Kleiser   
2  June 20, 1980  0.918919  0.499993   Irvin Kershner   
3   July 2, 1980  0.783784  0.092070     Jim Abrahams   
4  July 25, 1980  0.729730  0.044986     Harold Ramis   

                    writer            star         country  ...  \
0             Stephen King  Jack Nicholson  United Kingdom  ...   
1  Henry De Vere Stacpoole  Brooke Shields   United States  ...   
2           Leigh Brackett     Mark Hamill   United S

##### 'company' & 'country' column encoding

This code applies frequency encoding to the 'company' and 'country' columns by calculating how often each unique value appears in the dataset and mapping those frequencies back to the original data. 

In [8]:
# Calculate the frequency of each unique value in the 'company' column
company_freq = data_frame['company'].value_counts() / len(data_frame)
# Map the frequencies back to the original dataframe
data_frame['company'] = data_frame['company'].map(company_freq)

# Calculate the frequency of each unique value in the 'country' column
country_freq = data_frame['country'].value_counts() / len(data_frame)
# Map the frequencies back to the original dataframe
data_frame['country'] = data_frame['country'].map(country_freq)

#Print the results
print(data_frame[['company', 'country']])


       company   country
0     0.043966  0.106681
1     0.043834  0.719039
2     0.001320  0.719039
3     0.042250  0.719039
4     0.007922  0.719039
...        ...       ...
7569  0.000132  0.004621
7570  0.000132  0.719039
7571  0.000132  0.719039
7572  0.000132  0.719039
7573  0.000264  0.719039

[7574 rows x 2 columns]


##### Convert 'released' to a date format

This code converts the 'released' column to datetime format, ensuring consistency in date representation. Rows with invalid or missing dates are removed to maintain data integrity, preventing errors in time-based feature engineering.

In [9]:
# Convert the 'released' column to datetime format
data_frame['released'] = pd.to_datetime(data_frame['released'], errors='coerce')

# Check for any rows that could not be converted
data_frame = data_frame.dropna(subset=['released'])
data_frame.isnull().sum()

# Display the first few rows of the DataFrame to verify the conversion
print(data_frame[['released']])

       released
0    1980-06-13
1    1980-07-02
2    1980-06-20
3    1980-07-02
4    1980-07-25
...         ...
7569 2020-08-28
7570 2020-04-17
7571 2020-06-03
7572 2020-02-07
7573 2020-03-03

[7519 rows x 1 columns]


#### Creating Domain-Specific Features

Domain knowledge is about understanding the domain or subject area of the data. 


##### Time Based Features

This code extracts the month and quarter from each movie’s release date to create new features: `release_month` and `release_quarter`. These features help capture seasonal trends, as movies released during certain times of the year (like summer or holidays) may perform better. Including them allows the model to learn patterns related to release timing that could impact a movie’s success.

In [10]:
# Extracting month and quarter from release date
data_frame['release_month'] = pd.to_datetime(data_frame['released']).dt.month
data_frame['release_quarter'] = pd.to_datetime(data_frame['released']).dt.quarter

print(data_frame[['release_month', 'release_quarter']])

      release_month  release_quarter
0                 6                2
1                 7                3
2                 6                2
3                 7                3
4                 7                3
...             ...              ...
7569              8                3
7570              4                2
7571              6                2
7572              2                1
7573              3                1

[7519 rows x 2 columns]


##### Sequel

This code uses a regular expression to identify potential sequels in movie titles by searching for common sequel indicators such as numbers, Roman numerals, or words like Return, Final, or Reboot. It creates a new binary column `is_sequel`, where 1 indicates a sequel and 0 otherwise. This feature can help the model recognize whether a film is part of a franchise, which may influence its performance due to established audiences.

In [11]:
#regex for detecting sequels
sequel_pattern = r'(?i)\b(?:2|3|4|5|II|III|IV|V|VI|VII|VIII|IX|X|Part \d|Chapter \d|Revenge|Return|Resurrection|Legacy|Reboot|Rise|Reloaded|Evolution|Final|The New)\b'

# Create a new column 'is_sequel' based on the regex pattern
data_frame['is_sequel'] = data_frame['name'].apply(lambda x: bool(re.search(sequel_pattern, str(x)))).astype(int)

# Display the first few rows of the DataFrame to verify the new column
print(data_frame[['name', 'is_sequel']].head(10))


                                             name  is_sequel
0                                     The Shining          0
1                                 The Blue Lagoon          0
2  Star Wars: Episode V - The Empire Strikes Back          1
3                                       Airplane!          0
4                                      Caddyshack          0
5                                 Friday the 13th          0
6                              The Blues Brothers          0
7                                     Raging Bull          0
8                                     Superman II          1
9                                 The Long Riders          0


#### Save the wrangled and engineered data to CSV

In [12]:
data_frame.to_csv('../Model_Training/model_ready_data.csv', index=False)